# Model 3: G/G/2
Based on Lab 5 code

In [116]:
import pandas as pd
import numpy
import numpy as np

In [117]:
#Load Data as arrays
RA = pd.read_csv('RANorm.csv', names=['arrival'], usecols=[1])
RD = pd.read_csv('RDNorm.csv', names=['departure'], usecols=[1])
LA = pd.read_csv('LANorm.csv', names=['arrival'], usecols=[1])
LD = pd.read_csv('LDNorm.csv', names=['departure'], usecols=[1])

# Convertime string to seconds
RD["departure"] = pd.to_timedelta(RD["departure"].radd("00:")).dt.total_seconds()
RA["arrival"] = pd.to_timedelta(RA["arrival"].radd("00:")).dt.total_seconds()
LD["departure"] = pd.to_timedelta(LD["departure"].radd("00:")).dt.total_seconds()
LA["arrival"] = pd.to_timedelta(LA["arrival"].radd("00:")).dt.total_seconds()

# Merge into right and left Dataframes
right = pd.concat([RA, RD], axis=1)
left = pd.concat([LA, LD], axis=1)

# add new column "serving time", the difference between departure and the max of arrival or the previous departure
serving_time = []
for i in range(len(right)):
    if i == 0:
        serving_time.append(right['departure'][i] - right['arrival'][i])
    else:
        serving_time.append(right['departure'][i] - max(right['arrival'][i], right['departure'][i-1]))
right['serving_time'] = serving_time
right['serving_time'] = right['serving_time'].clip(lower=0)

serving_time = []
for i in range(len(left)):
    if i == 0:
        serving_time.append(left['departure'][i] - left['arrival'][i])
    else:
        serving_time.append(left['departure'][i] - max(left['arrival'][i], left['departure'][i-1]))
left['serving_time'] = serving_time
left['serving_time'] = left['serving_time'].clip(lower=0)

# get the interarrival time for the system (right + left)
interarrival = []

combined = pd.concat([right, left], axis=0)
combined = combined.sort_values(by=['arrival'])
combined = combined.reset_index(drop=True)

for i in range(len(combined)):
    if i == 0:
        interarrival.append(combined['arrival'][i])
    else:
        interarrival.append(combined['arrival'][i] - combined['arrival'][i-1])
combined['interarrival'] = interarrival
combined['interarrival'] = combined['interarrival'].clip(lower=0)

In [118]:
def empirical_cdf(data):
    n = len(data)
    x = np.sort(data)
    y = np.arange(1,n+1)/n
    return x, y

def draw_empirical(data, r):
    """one draw (for given r ~ U(0,1)) from the empirical cdf based on data"""
    x, y = empirical_cdf(data)
    return x[numpy.argmax(y>r)]

In [119]:
"""(q4.py) M/M/c queueing system with several monitors and multiple replications"""

from SimPy.Simulation import *
import random
import numpy
import math

In [120]:
def conf(L):
    """confidence interval"""
    lower = numpy.mean(L) - 1.96*numpy.std(L)/math.sqrt(len(L))
    upper = numpy.mean(L) + 1.96*numpy.std(L)/math.sqrt(len(L))
    return lower, upper

In [121]:
class Source(Process):
    """generate random arrivals"""
    def run(self, N):
        for i in range(N):
            a = Arrival(str(i))
            activate(a, a.run())
            r = random.random()
            t = draw_empirical(combined['interarrival'], r)
            yield hold, self, t

In [122]:
class Arrival(Process):
    """an arrival"""
    n = 0  # class variable (number in system)

    def run(self):
        # Event: arrival
        Arrival.n += 1   # number in system
        arrivetime = now()
        G.numbermon.observe(Arrival.n)
        if (Arrival.n>0):
            G.busymon.observe(1)
        else:
            G.busymon.observe(0)

        yield request, self, G.server
        # ... waiting in queue for server to be empty (delay) ...
      
        # Event: service begins
        r = random.random()
        t = draw_empirical(combined['serving_time'], r)
      
        yield hold, self, t
        # ... now being served (activity) ...
      
        # Event: service ends
        yield release, self, G.server # let go of server (takes no simulation time)
        Arrival.n -= 1
        G.numbermon.observe(Arrival.n)
        if (Arrival.n>0):
            G.busymon.observe(1)
        else:
            G.busymon.observe(0)
        delay = now()-arrivetime
        G.delaymon.observe(delay)

In [123]:
class G:
    # server = 'dummy'
    # delaymon = 'Monitor'
    # numbermon = 'Monitor'
    # busymon = 'Monitor'
    def __init__(self, c):
        self.server = Resource(capacity=1, monitored=True)
        self.delaymon = Monitor()
        self.numbermon = Monitor()
        self.busymon = defaultdict(Monitor)
        self.c = c

In [124]:
def model(c, N, maxtime, rvseed):
    # setup
    initialize()
    random.seed(rvseed)
    G.server = Resource(c)
    G.delaymon = Monitor()
    G.numbermon = Monitor()
    G.busymon = Monitor()
    
    Arrival.n = 0
   
    # simulate
    s = Source('Source')
    activate(s, s.run(N))
    simulate(until=maxtime)

    # gather performance measures
    W = G.delaymon.mean()
    L = G.numbermon.timeAverage()
    B = G.busymon.timeAverage()
    return W, L, B

In [125]:
## The serving distribution is described by 0.9*expon.pdf(x, 0, 1.5) + 0.1*norm.pdf(x, 60, 7)

def serve_dist():
    if random.random() < 0.95:
        return random.expovariate(1/1.5)
    else:
        return random.normalvariate(60, 2)

## Experiment ----------------
allW = []
allL = []
allB = []
allLambdaEffective = []
for k in range(50):
    seed = 123*k
    result = model(c=2, N=10000, maxtime=200000, rvseed=seed)
    allW.append(result[0])
    allL.append(result[1])
    allB.append(result[2])
    allLambdaEffective.append(result[1]/result[0])

In [126]:
print("Estimate of W:", numpy.mean(allW))
print("Conf int of W:", conf(allW))
print("Estimate of L:", numpy.mean(allL))
print("Conf int of L:", conf(allL))
print("Estimate of B:", numpy.mean(allB))
print("Conf int of B:", conf(allB))
print("Estimate of LambdaEffective:", numpy.mean(allLambdaEffective))
print("Conf int of LambdaEffective:", conf(allLambdaEffective))

Estimate of W: 35.126661840000494
Conf int of W: (32.83981777025352, 37.41350590974747)
Estimate of L: 14.182300896180914
Conf int of L: (13.230964230711708, 15.13363756165012)
Estimate of B: 0.8609109662623194
Conf int of B: (0.8555238138162881, 0.8662981187083508)
Estimate of LambdaEffective: 0.403212414359209
Conf int of LambdaEffective: (0.40147491220736337, 0.40494991651105466)
